In [4]:

import gzip
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Ruta a la carpeta donde están los archivos .CAT.gz
carpeta_urbana = Path("C:\\Users\\joselef\\Downloads\\2025042214341852038_PETICION_DESCARGA_CAT\\03_U_17012025_CAT\\03_U_17012025_CAT")  # <- CAMBIA ESTO
carpeta_rustica = Path("C:\\Users\\joselef\\Downloads\\2025042307514431841_PETICION_DESCARGA_CAT\\03_R_17012025_CAT\\03_R_17012025_CAT") # CAMBIA ESTO


# Funciones de extracción por tipo de registro
def procesar_cat11(lineas):
    registros = []
    for linea in lineas:
        if linea.startswith("11"):
            try:
                registros.append({
                    "delegacion": linea[23:25].strip(),
                    "municipio_dgc": linea[25:28].strip(),
                    "ref_catastral": linea[30:44].strip(),
                    "provincia": linea[52:77].strip(),
                    "municipio": linea[83:123].strip(),
                    "via": linea[163:188].strip(),
                    "num_pol": linea[188:192].strip(),
                    "superficie_finca": float(linea[295:305].replace(",", ".")),
                    "sup_const_total": float(linea[305:312].replace(",", ".")),
                    "sup_const_sobre": float(linea[312:319].replace(",", ".")),
                    "sup_const_bajo": float(linea[319:326].replace(",", ".")),
                    "coor_x": float(linea[333:342].replace(",", ".")),
                    "coor_y": float(linea[342:352].replace(",", ".")),
                    "epsg": linea[666:676].strip()
                })
            except Exception as e:
                print(f"[CAT11] Error: {e}\n{linea}")
    return pd.DataFrame(registros)

def procesar_cat13(lineas):
    registros = []
    for linea in lineas:
        if linea.startswith("13"):
            try:
                registros.append({
                    "ref_catastral": linea[30:44].strip(),
                    "codigo_uc": linea[44:48].strip(),
                    "provincia": linea[52:77].strip(),
                    "municipio": linea[83:123].strip(),
                    "via": linea[163:188].strip(),
                    "num_pol": linea[188:192].strip(),
                    "anio_construccion": linea[295:299].strip(),
                    "sup_ocupada": float(linea[300:307].replace(",", "."))
                })
            except Exception as e:
                print(f"[CAT13] Error: {e}\n{linea}")
    return pd.DataFrame(registros)

def procesar_cat14(lineas):
    registros = []
    for linea in lineas:
        if linea.startswith("14"):
            try:
                registros.append({
                    "ref_catastral": linea[30:44].strip(),
                    "codigo_construccion": linea[44:48].strip(),
                    "codigo_uc": linea[54:58].strip(),
                    "destino": linea[70:73].strip(),
                    "anio_reforma": linea[74:78].strip(),
                    "sup_total": float(linea[83:90].replace(",", "."))
                })
            except Exception as e:
                print(f"[CAT14] Error: {e}\n{linea}")
    return pd.DataFrame(registros)

def procesar_cat15(lineas):
    registros = []
    for linea in lineas:
        if linea.startswith("15"):
            try:
                registros.append({
                    "ref_catastral": linea[30:44].strip(),
                    "num_cargo": linea[44:48].strip(),
                    "provincia": linea[94:119].strip(),
                    "municipio": linea[125:165].strip(),
                    "via": linea[205:230].strip(),
                    "num_pol": linea[230:234].strip(),
                    "sup_constructiva": float(linea[441:451].replace(",", ".")),
                    "sup_suelo": float(linea[451:461].replace(",", ".")),
                    "coef_propiedad": float(linea[461:470].replace(",", "."))
                })
            except Exception as e:
                print(f"[CAT15] Error: {e}\n{linea}")
    return pd.DataFrame(registros)

def procesar_cat16(lineas):
    registros = []
    for linea in lineas:
        if linea.startswith("16"):
            try:
                registros.append({
                    "ref_catastral": linea[30:44].strip(),
                    "elemento": linea[44:48].strip(),
                    "reparto1": linea[50:54].strip(),
                    "porcentaje1": float(linea[54:60].replace(",", "."))
                })
            except Exception as e:
                print(f"[CAT16] Error: {e}\n{linea}")
    return pd.DataFrame(registros)

def procesar_cat17(lineas):
    registros = []
    for linea in lineas:
        if linea.startswith("17"):
            try:
                registros.append({
                    "ref_catastral": linea[30:44].strip(),
                    "codigo_subparcela": linea[44:48].strip(),
                    "tipo": linea[54].strip(),
                    "sup_subparcela": float(linea[55:65].replace(",", ".")),
                    "cultivo": linea[67:107].strip()
                })
            except Exception as e:
                print(f"[CAT17] Error: {e}\n{linea}")
    return pd.DataFrame(registros)

# Lista de funciones a ejecutar
tipos = {
    "CAT11_fincas.csv": procesar_cat11,
    "CAT13_uc.csv": procesar_cat13,
    "CAT14_const.csv": procesar_cat14,
    "CAT15_inmuebles.csv": procesar_cat15,
    "CAT16_reparto.csv": procesar_cat16,
    "CAT17_cultivos.csv": procesar_cat17,
}

# Unificar archivos de urbana y rústica
archivos_cat = list(carpeta_urbana.glob("*.CAT.gz")) + list(carpeta_rustica.glob("*.CAT.gz"))

# Proceso igual que antes
for nombre_archivo, funcion in tipos.items():
    todos_los_datos = []

    def procesar_archivo(archivo):
        with gzip.open(archivo, 'rt', encoding='cp1252') as f:
            lineas = f.readlines()
            return funcion(lineas)

    with ThreadPoolExecutor() as executor:
        for df in tqdm(executor.map(procesar_archivo, archivos_cat), total=len(archivos_cat), desc=f"Procesando {nombre_archivo}"):
            if not df.empty:
                todos_los_datos.append(df)

    if todos_los_datos:
        df_final = pd.concat(todos_los_datos, ignore_index=True)
        df_final.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
        print(f"Exportado {nombre_archivo} con {len(df_final)} registros.")
    else:
        print(f"No se encontraron datos para {nombre_archivo}.")


Procesando CAT11_fincas.csv: 100%|███████████████████████████████████████████████████| 282/282 [01:20<00:00,  3.52it/s]


✅ Exportado CAT11_fincas.csv con 931224 registros.


Procesando CAT13_uc.csv: 100%|███████████████████████████████████████████████████████| 282/282 [01:14<00:00,  3.78it/s]


✅ Exportado CAT13_uc.csv con 998626 registros.


Procesando CAT14_const.csv: 100%|████████████████████████████████████████████████████| 282/282 [01:23<00:00,  3.37it/s]


...

✅ Exportado CAT15_inmuebles.csv con 2577618 registros.


Procesando CAT16_reparto.csv: 100%|██████████████████████████████████████████████████| 282/282 [01:08<00:00,  4.09it/s]


✅ Exportado CAT16_reparto.csv con 141528 registros.


Procesando CAT17_cultivos.csv: 100%|█████████████████████████████████████████████████| 282/282 [01:12<00:00,  3.86it/s]


✅ Exportado CAT17_cultivos.csv con 759602 registros.
